In [1]:
import numpy as np 
import pandas as pd 
from sklearn.linear_model import LinearRegression, Ridge


In [2]:
p = 7
n = 10
min_cor = 0
max_cor = 0.1

true_betas = np.array([0,1,2,3,4,5,6])

In [3]:
def get_sim_data(p, n, min_cor, max_cor, true_betas):
    
    sd_vec = np.ones(p) 
    mean = np.zeros(p)
    cor_matrix = np.zeros((p,p))

    correlation = np.random.uniform(min_cor, max_cor, int(p * (p - 1) / 2))
    cor_matrix[np.triu_indices(p, 1)] = correlation
    cor_matrix[np.tril_indices(p, -1)] = cor_matrix.T[np.tril_indices(p, -1)]
    np.fill_diagonal(cor_matrix, 1)


    D = np.diag(sd_vec)
    sigma = D.dot(cor_matrix).dot(D)

    X = np.random.multivariate_normal(mean, sigma, n)
    eps = np.random.normal(0, 1, n)

    y = X.dot(true_betas) + eps 
    
    y = pd.Series(y, name = "y")
    
    column_names = []
    
    for value in range(1, p + 1): 
        
        column = f"X_{value}"
        column_names.append(column)
        
    
    X = pd.DataFrame(X, columns = column_names)
    
    df = pd.concat([y, X], axis = 1)
    
    return y, X, df



In [4]:
y, X, df = get_sim_data(p, n, min_cor, max_cor, true_betas)

In [5]:
df

,y,X_1,X_2,X_3,X_4,X_5,X_6,X_7
0,6.700304,0.229795,1.418748,0.293733,0.227455,0.049370,0.523640,0.395193
1,-8.838713,0.826909,-2.236812,-1.045868,-0.046392,0.115875,-0.560652,-0.478897
2,1.300872,-3.255779,1.512920,-0.027390,0.559986,-0.413191,0.135853,-0.007789
3,-2.338110,0.467921,0.745094,-0.619800,0.148451,1.089779,-0.994641,-0.124006
4,-4.388244,-0.426627,0.001085,0.759819,-1.407267,-0.865989,0.759800,-0.342128
5,-11.556351,-0.609506,0.281355,-0.150776,-1.514042,0.536812,-0.932207,-0.521598
6,-19.457491,-1.059193,0.280662,0.025637,-1.490183,-1.081338,0.523433,-1.855455
7,-14.132090,0.388177,0.727049,-1.357356,-2.481721,-0.391448,0.019262,-0.571806
8,5.318901,-0.018340,0.357609,0.213398,1.906580,-1.275371,0.561506,0.140701
9,5.143152,-0.031630,-0.811497,0.446632,-0.674455,1.030167,0.892548,-0.395850


In [6]:
reg = LinearRegression().fit(X, y)

In [7]:
reg.coef_


array([-0.13525375,  0.19914559,  1.25999097,  2.8054732 ,  4.0200966 ,
        5.84810088,  7.54052348])

In [8]:
"""Zero multicollinearity.
   n = 200; p = 2, 5, 50, 100, 150, 200, 210, 250. 
"""
np.random.seed(123)

n_sim = 30
p_sim = np.array([2, 28, 30, 35])
dataframes = len(p_sim)
min_corr_sim = 0
max_corr_sim = 0 
iterations_sim = 500

matrix_betas_final = []
matrix_betas_mean = []
matrix_bias_sq = []
matrix_betas_var_mean = []
matrix_betas_var_final = []


for m in range(dataframes): 
    
    
    for p in p_sim: 
    
        matrix_var = []
        matrix_betas = []
    
        for i in range(iterations_sim):
    
            true_betas_sim = np.repeat(5, p)
    
            y, X, df = get_sim_data(p, n_sim, min_corr_sim, max_corr_sim, true_betas_sim) 
    
            var_cov = np.linalg.inv(np.dot(X.T, X))
            var_betas = var_cov.diagonal() 
        
            #reg = LinearRegression().fit(X, y)
            betas = np.dot(np.dot(np.linalg.inv(np.dot(X.T, X)), X.T), y)
        
            #matrix_betas.append(reg.coef_)
            matrix_betas.append(betas)
        
            matrix_var.append(var_betas)
        
            X_names = []
            beta_names = []
    
            for value in range(1, p + 1): 
            
                column_X = f"X_{value}"
                column_betas = f"beta_{value}"
                X_names.append(column_X)
                beta_names.append(column_betas)
        
            df_var = pd.DataFrame(matrix_var, columns = X_names)
            df_betas = pd.DataFrame(matrix_betas, columns = beta_names) 
        
            bias_squared = (df_betas.mean() - true_betas_sim)**2
        
        matrix_betas_final.append(df_betas)
        matrix_betas_mean.append(df_betas.mean())
        matrix_bias_sq.append(bias_squared)
        matrix_betas_var_mean.append(df_var.mean())
        matrix_betas_var_final.append(df_var)
    
    #print(df_var.mean()) 
    #print(df_betas.mean())
    #print(bias_squared)
    #print(df_var)

In [9]:
matrix_betas_var_mean[0]

X_1    0.036147
X_2    0.037652
dtype: float64

In [10]:
matrix_betas_var_mean[1]

X_1     0.905170
X_2     0.894537
X_3     0.846209
X_4     0.947683
X_5     0.914715
X_6     0.857703
X_7     0.926860
X_8     0.728121
X_9     0.809547
X_10    0.789987
X_11    0.756611
X_12    0.779219
X_13    0.922239
X_14    0.806494
X_15    0.905561
X_16    0.878548
X_17    0.903779
X_18    0.722353
X_19    0.817044
X_20    0.922587
X_21    0.813028
X_22    0.834893
X_23    0.883378
X_24    0.842640
X_25    0.819369
X_26    0.830576
X_27    0.898804
X_28    0.909811
dtype: float64

In [11]:
matrix_betas_var_mean[2]

X_1     1.405411e+06
X_2     2.061499e+04
X_3     1.553527e+04
X_4     4.477379e+05
X_5     9.174382e+04
X_6     2.776765e+06
X_7     1.714348e+04
X_8     4.412726e+05
X_9     9.133277e+05
X_10    3.290059e+06
X_11    6.355281e+04
X_12    3.866585e+05
X_13    1.978667e+05
X_14    1.713295e+06
X_15    1.030510e+06
X_16    1.235208e+06
X_17    2.998218e+05
X_18    9.515781e+05
X_19    1.840924e+06
X_20    2.916021e+06
X_21    3.332955e+06
X_22    5.552680e+05
X_23    4.594443e+05
X_24    1.782019e+05
X_25    3.528304e+06
X_26    1.984085e+05
X_27    1.446316e+07
X_28    5.981436e+03
X_29    2.581248e+06
X_30    7.506580e+05
dtype: float64

In [12]:
matrix_betas_var_mean[3]

X_1    -2.766848e+13
X_2    -3.320289e+13
X_3     3.246389e+13
X_4     6.095380e+12
X_5    -2.690809e+13
X_6    -3.741737e+13
X_7    -2.049798e+13
X_8    -4.448690e+13
X_9    -1.379433e+13
X_10   -2.104500e+13
X_11    1.082571e+13
X_12    2.780041e+12
X_13    9.565688e+11
X_14   -2.279610e+12
X_15    4.250038e+13
X_16    6.405215e+12
X_17   -1.033276e+13
X_18   -4.202050e+13
X_19   -1.930038e+13
X_20   -2.906674e+13
X_21    2.711959e+13
X_22    1.142487e+13
X_23   -9.667820e+13
X_24   -3.028840e+12
X_25   -3.052798e+13
X_26   -1.196195e+13
X_27    1.181959e+13
X_28   -2.375272e+13
X_29   -5.396132e+13
X_30   -1.359054e+13
X_31   -3.697549e+13
X_32   -1.530770e+13
X_33    6.374014e+12
X_34   -1.058066e+13
X_35    4.707655e+12
dtype: float64

In [8]:
"""Zero multicollinearity.
   n = 200; p = 2, 5, 50, 100, 150, 200, 210, 250. 
"""
np.random.seed(123)

n_sim = 30
p_sim = np.array([2, 28, 30, 35])
min_corr_sim = 0
max_corr_sim = 0 
iterations_sim = 10000

for p in p_sim: 
    
    matrix_var = []
    matrix_betas = []
    
    for i in range(iterations_sim):
    
        true_betas_sim = np.repeat(5, p)
    
        y, X, df = get_sim_data(p, n_sim, min_corr_sim, max_corr_sim, true_betas_sim) 
    
        var_cov = np.linalg.inv(np.dot(X.T, X))
        var_betas = var_cov.diagonal() 
        
        #reg = LinearRegression().fit(X, y)
        betas = np.dot(np.dot(np.linalg.inv(np.dot(X.T, X)), X.T), y)
        
        #matrix_betas.append(reg.coef_)
        matrix_betas.append(betas)
        
        matrix_var.append(var_betas)
        
        X_names = []
        beta_names = []
    
        for value in range(1, p + 1): 
            
            column_X = f"X_{value}"
            column_betas = f"beta_{value}"
            X_names.append(column_X)
            beta_names.append(column_betas)
        
        df_var = pd.DataFrame(matrix_var, columns = X_names)
        df_betas = pd.DataFrame(matrix_betas, columns = beta_names) 
        
        bias_squared = (df_betas.mean() - true_betas_sim)**2
        
    #print(df_var.mean()) 
    #print(df_betas.mean())
    print(bias_squared)
    #print(df_var)

beta_1    0.000001
beta_2    0.000010
dtype: float64
beta_1     1.277691e-06
beta_2     9.295848e-07
beta_3     6.057674e-05
beta_4     3.777720e-04
beta_5     1.356946e-08
beta_6     1.126223e-05
beta_7     2.091453e-04
beta_8     1.321072e-06
beta_9     2.360325e-04
beta_10    1.766855e-04
beta_11    2.167602e-05
beta_12    3.560336e-04
beta_13    6.823048e-05
beta_14    1.101843e-04
beta_15    1.192075e-04
beta_16    1.165771e-04
beta_17    1.309346e-05
beta_18    1.963390e-04
beta_19    1.273189e-04
beta_20    1.272697e-04
beta_21    6.564113e-05
beta_22    4.956159e-05
beta_23    5.989386e-05
beta_24    1.456125e-04
beta_25    2.063340e-04
beta_26    7.138961e-05
beta_27    1.855968e-04
beta_28    7.513351e-06
dtype: float64
beta_1     0.509496
beta_2     0.116442
beta_3     0.374170
beta_4     0.539417
beta_5     0.169819
beta_6     0.309164
beta_7     4.250944
beta_8     2.004033
beta_9     0.088176
beta_10    0.020643
beta_11    1.288161
beta_12    1.943307
beta_13    0.084237


In [9]:
np.random.seed(159)

p = 198
n = 200
true_betas = np.repeat(5, p)
min_cor = 0 
max_cor = 0 
alpha = 0.001


y, X, df = get_sim_data(p, n, min_cor, max_cor, true_betas)

n, m = X.shape
I = np.identity(m)


ols = np.dot(np.dot(np.linalg.inv(np.dot(X.T, X)), X.T), y)

ols_var_cov = np.linalg.inv(np.dot(X.T, X))
ols_var = ols_var_cov.diagonal()

alphas = np.array([0, 0.0001, 0.001, 0.01, 0.1, 0.2, 0.5, 1, 2, 3, 5, 10, 15, 20])

matr_var = []
matr_beta = []

for a in alphas: 

    ridge_beta = np.dot(np.dot(np.linalg.inv(np.dot(X.T, X) + a * I), X.T), y)
    matr_beta.append(ridge_beta)
    
    ridge_var_cov = np.dot(np.dot(np.linalg.inv(np.dot(X.T, X) + a * I), np.dot(X.T, X)), np.linalg.inv(np.dot(X.T, X) + a * I))
    ridge_var = ridge_var_cov.diagonal()
    matr_var.append(ridge_var)

beta_var_names = []
ridge_beta_names = []
    
for value in range(1, p + 1): 
    
    column_betas_var = f"beta_var_{value}"
    column_betas = f"beta_{value}"
    beta_var_names.append(column_betas_var)
    ridge_beta_names.append(column_betas)


In [10]:
beta_var = pd.DataFrame(matr_var, columns = beta_var_names)
beta_var["alpha"] = alphas
beta_var.set_index("alpha")    

,beta_var_1,beta_var_2,beta_var_3,beta_var_4,beta_var_5,beta_var_6,beta_var_7,beta_var_8,beta_var_9,beta_var_10,...,beta_var_189,beta_var_190,beta_var_191,beta_var_192,beta_var_193,beta_var_194,beta_var_195,beta_var_196,beta_var_197,beta_var_198
alpha,,,,,,,,,,,,,,,,,,,,,
0.0000,2.427521,0.663690,1.027757,1.741210,8.652040,0.705819,1.167791,0.209445,0.475403,1.731693,...,0.464720,1.398587,0.242995,4.707146,5.263580,0.286077,0.432695,0.264678,2.052473,2.387135
0.0001,2.268247,0.638268,1.015365,1.677230,8.118124,0.696873,1.102453,0.208107,0.460090,1.643960,...,0.463072,1.368871,0.240256,4.421376,4.932370,0.284450,0.421808,0.254588,1.924919,2.358828
0.0010,1.365506,0.489142,0.920092,1.289592,5.063371,0.636114,0.727835,0.198151,0.372755,1.134611,...,0.451391,1.173477,0.223555,2.786644,3.046089,0.271885,0.354129,0.194701,1.197613,2.140829
0.0100,0.233427,0.236424,0.484755,0.491798,0.885691,0.419897,0.204628,0.153566,0.250316,0.344555,...,0.389761,0.584647,0.184876,0.552214,0.567427,0.210356,0.193110,0.085665,0.232947,1.135968
0.1000,0.071878,0.109102,0.109703,0.102415,0.102458,0.134093,0.083359,0.087886,0.147321,0.092087,...,0.171041,0.131156,0.122357,0.108965,0.119170,0.112952,0.080600,0.037393,0.067317,0.221369
0.2000,0.052158,0.083912,0.072989,0.070817,0.071605,0.079757,0.069978,0.068406,0.105920,0.066938,...,0.106042,0.082435,0.096294,0.079727,0.078798,0.084015,0.060815,0.032334,0.055253,0.124536
0.5000,0.033333,0.055552,0.044977,0.045025,0.047987,0.041008,0.053829,0.046834,0.062048,0.044307,...,0.054870,0.048374,0.064342,0.053194,0.045295,0.051499,0.040217,0.026727,0.040510,0.059026
1.0000,0.024475,0.038056,0.032094,0.032398,0.033820,0.026810,0.041855,0.034336,0.040391,0.032550,...,0.035335,0.034211,0.044444,0.037953,0.030069,0.033482,0.029148,0.022248,0.029858,0.035925
2.0000,0.018267,0.024648,0.022964,0.023233,0.022680,0.018520,0.030327,0.024222,0.026329,0.023708,...,0.023922,0.024243,0.028791,0.026096,0.020006,0.021458,0.021236,0.017511,0.020976,0.023199


In [11]:
ridge_beta = pd.DataFrame(matr_beta, columns = ridge_beta_names)
ridge_beta["alpha"] = alphas
ridge_beta.set_index("alpha")

,beta_1,beta_2,beta_3,beta_4,beta_5,beta_6,beta_7,beta_8,beta_9,beta_10,...,beta_189,beta_190,beta_191,beta_192,beta_193,beta_194,beta_195,beta_196,beta_197,beta_198
alpha,,,,,,,,,,,,,,,,,,,,,
0.0000,1.431469,5.807467,3.926156,2.236071,0.072503,7.624101,7.220508,5.102859,4.031463,1.447353,...,4.220671,5.882952,5.872391,8.546182,1.462723,4.922933,4.285627,4.374181,1.437395,5.958386
0.0001,1.551141,5.760691,3.915839,2.308962,0.281415,7.600136,7.147344,5.096939,4.067559,1.535833,...,4.228182,5.847133,5.857172,8.392787,1.628563,4.918490,4.315292,4.401010,1.545047,5.926595
0.0010,2.338443,5.451785,3.858992,2.793782,1.640578,7.438045,6.670658,5.058287,4.303149,2.121703,...,4.274300,5.623312,5.755458,7.393551,2.708076,4.891785,4.513975,4.574284,2.255773,5.701470
0.0100,4.108101,4.753317,3.948965,3.958287,4.418631,6.995541,5.674130,4.953750,4.783558,3.498915,...,4.297046,5.352289,5.495505,5.317081,4.910800,4.871660,5.022244,4.908232,3.901663,4.875434
0.1000,4.758536,4.820550,4.730247,4.126283,4.941148,6.713544,5.227479,4.417448,4.578426,3.955284,...,3.730093,6.021328,5.257863,4.616056,5.022495,4.828649,5.279141,4.975867,4.645925,3.368100
0.2000,4.744429,5.063842,5.059990,3.865562,4.874784,6.685456,5.036722,4.032856,4.343217,3.838261,...,3.357149,6.355777,5.131745,4.428845,4.718949,4.763723,5.262315,4.970348,4.674205,2.772379
0.5000,4.603572,5.383110,5.408980,3.515787,4.777159,6.622313,4.653210,3.432380,4.030708,3.569397,...,2.814711,6.746293,4.826490,4.070268,4.211372,4.675612,5.206396,4.864943,4.557786,2.034509
1.0000,4.401496,5.517441,5.474450,3.388484,4.722273,6.515898,4.239832,2.938190,3.873702,3.321878,...,2.446932,6.900269,4.468958,3.700577,3.778271,4.617793,5.180779,4.629816,4.349650,1.661489
2.0000,4.120503,5.530916,5.266060,3.435371,4.661601,6.315323,3.701556,2.370742,3.801054,3.054539,...,2.126772,6.891306,4.039137,3.245606,3.289118,4.547337,5.215155,4.207783,4.066485,1.514109


$$
V\left(\hat{\boldsymbol{\beta}}^{*}\right)=\sigma^{2}\left[\mathbf{X}^{\prime} \mathbf{X}+k \mathbf{I}\right]^{-1}\left(\mathbf{X}^{\prime} \mathbf{X}\right)\left[\mathbf{X}^{\prime} \mathbf{X}+k \mathbf{I}\right]^{-1}.
$$

In [13]:
np.random.seed(159)

p = 198
n = 200
min_cor = 0 
max_cor = 0 
iterations_sim = 5


#alphas = np.array([0, 0.0001, 0.001, 0.01, 0.1, 0.2, 0.5, 1, 2, 3, 5, 10, 15, 20])
alphas = np.array([0, 0.0001])


for a in alphas: 
    
    matr_var = []
    matr_beta = []
    
    for i in range(iterations_sim):
        
        true_betas_sim = np.repeat(5, p)
    
        y, X, df = get_sim_data(p, n, min_cor, max_cor, true_betas_sim) 
        
        n, m = X.shape
        I = np.identity(m)

        ridge_beta = np.dot(np.dot(np.linalg.inv(np.dot(X.T, X) + a * I), X.T), y)
        matr_beta.append(ridge_beta)
        df_ridge_betas = pd.DataFrame(matr_beta).mean()
        
        ridge_var_cov = np.dot(np.dot(np.linalg.inv(np.dot(X.T, X) + a * I), np.dot(X.T, X)), np.linalg.inv(np.dot(X.T, X) + a * I))
        ridge_var = ridge_var_cov.diagonal()
        matr_var.append(ridge_var)
        df_ridge_var = pd.DataFrame(matr_var).mean()
        
    
    print(df_ridge_betas)
    #print(df_ridge_var)
       
    
        

    #beta_var_names = []
    #ridge_beta_names = []
    
    #for value in range(1, p + 1): 
    
        #column_betas_var = f"beta_var_{value}"
        #column_betas = f"beta_{value}"
        #beta_var_names.append(column_betas_var)
        #ridge_beta_names.append(column_betas)




0      4.635122
1      5.729616
2      5.058337
3      4.428545
4      3.234887
         ...   
193    6.171077
194    5.741595
195    4.427468
196    4.298631
197    4.852898
Length: 198, dtype: float64
0      4.335456
1      5.466356
2      5.149386
3      4.969014
4      5.489306
         ...   
193    4.835337
194    5.127192
195    4.673648
196    5.121645
197    4.933246
Length: 198, dtype: float64


In [14]:
#pd.DataFrame(matr_beta)